In [1]:
import json
import pandas as pd
import numpy as np
from utils.data_helper import get_markable_dataframe, get_embedding_variables
from model_builders.coreference_classifier import CoreferenceClassifierModelBuilder
from functools import reduce
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from utils.clusterers import BestFirstClusterer, get_anaphora_scores_by_antecedent, ClosestFirstClusterer
from utils.scorers import MUCScorer, B3Scorer, AverageScorer
from utils.data_structures import UFDS

In [2]:
embedding_indexes_file_path = 'helper_files/embedding/embedding_indexes.txt'
indexed_embedding_file_path = 'helper_files/embedding/indexed_embedding.txt'

word_vector, embedding_matrix, idx_by_word, word_by_idx = get_embedding_variables(embedding_indexes_file_path, indexed_embedding_file_path)

In [11]:
markables_original_singletons = get_markable_dataframe("data/testing/markables.csv", word_vector, idx_by_word)
markables_predicted_singletons = get_markable_dataframe("data/testing/markables_with_predicted_singleton.csv", word_vector, idx_by_word)

markables_original_singletons.head()

,id,text,is_pronoun,entity_type,is_proper_name,is_first_person,previous_words,next_words,is_singleton
0,1916,"[1263, 1264, 1968, 1395]",0,"[0, 0, 0, 0, 1, 0, 1, 0, 0, 0]",1,0,[],"[999, 379, 1161, 213, 27, 1263, 1969, 1188, 14...","[1.0, 0.0]"
1,1917,[213],1,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]",0,0,"[1263, 1264, 1968, 1395, 999, 379, 1161]","[27, 1263, 1969, 1188, 1470, 25, 1161, 63, 424...","[1.0, 0.0]"
2,1918,"[1263, 1969, 1188]",0,"[0, 0, 1, 0, 1, 0, 0, 0, 0, 0]",1,0,"[1263, 1264, 1968, 1395, 999, 379, 1161, 213, 27]","[1470, 25, 1161, 63, 424, 1223, 25, 1415, 1161...","[1.0, 0.0]"
3,1919,"[1470, 25, 1161]",0,"[0, 0, 0, 0, 0, 1, 1, 0, 0, 0]",0,0,"[1968, 1395, 999, 379, 1161, 213, 27, 1263, 19...","[63, 424, 1223, 25, 1415, 1161, 876, 344, 213,...","[0.0, 1.0]"
4,1920,[424],0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]",0,0,"[1161, 213, 27, 1263, 1969, 1188, 1470, 25, 11...","[1223, 25, 1415, 1161, 876, 344, 213, 406, 122...","[0.0, 1.0]"


In [44]:
def get_markable_text(idx):
    return ' '.join([word_by_idx[x] for x in markables_original_singletons[markables_original_singletons['id'] == idx].text.values[0]])

def get_markable_previous_words(idx):
    return ' '.join([word_by_idx[x] for x in markables_original_singletons[markables_original_singletons['id'] == idx].previous_words.values[0]])

def get_markable_next_words(idx):
    return ' '.join([word_by_idx[x] for x in markables_original_singletons[markables_original_singletons['id'] == idx].next_words.values[0]])

def get_markable(idx):
    return f'{get_markable_previous_words(idx)} [{get_markable_text(idx).upper()}] {get_markable_next_words(idx)}'.strip()

In [45]:
right_predicted_markables = markable_predicted_singletons[markables_predicted_singletons['is_singleton'].map(lambda x: x[0]) == markables_original_singletons['is_singleton'].map(lambda x: x[0])]
wrong_predicted_markables = markable_predicted_singletons[markables_predicted_singletons['is_singleton'].map(lambda x: x[0]) != markables_original_singletons['is_singleton'].map(lambda x: x[0])]

# Analysis of Wrongly Predicted Markables

## Label: singleton, predicted: non-singleton

In [52]:
false_negatives = wrong_predicted_markables[wrong_predicted_markables['is_singleton'].map(lambda x: x[0]) == 1]
false_negative_texts = [get_markable(idx) for idx in false_negatives['id']]

In [56]:
for i, idx, false_negative_text in zip(range(1, len(false_negatives) + 1), false_negatives['id'], false_negative_texts):
    print(f'{i}. {idx}. {false_negative_text}')

1. 1923. <angka> saham dengan harga rp <angka> per saham atau total [NYA] senilai rp <angka> miliar kepada exedy corporation yang berkedudukan di
2. 1926. senilai rp <angka> miliar kepada exedy corporation yang berkedudukan di [JEPANG] kami telah menandatangani perjanjian pengikatan untuk penjualan seluruh saham milik
3. 1946. sisi direksi komisaris dan pemegang saham utama lagi pula nilai [NYA] tidak cukup material yakni kurang <angka> persen dari pendapatan dan
4. 1966. tanggal berlaku dia menambahkan permohonan pembebasan bm harus diajukan kepada [DIRJEN BEA CUKAI DEPKEU] dengan melampirkan antara lain npwp ; surat izin usaha dari
5. 2253. pada maret <angka> mencapai <angka> persen demikian dikemukakan deputi gubernur [BANK INDONESIA] bi hartadi a sarwono di jakarta rabu ia mengatakan persentase
6. 2254. <angka> mencapai <angka> persen demikian dikemukakan deputi gubernur bank indonesia [BI] hartadi a sarwono di jakarta rabu ia mengatakan persentase tersebut
7. 2268. hartadi dalam a

In [88]:
data_to_analyze =  false_negatives

print(f'"nya": {len(list(filter(lambda x: get_markable_text(x) == "nya", data_to_analyze["id"])))}')
print(f'proper name: {len(data_to_analyze[data_to_analyze["is_proper_name"] == 1])}')
print(f'pronoun: {len(data_to_analyze[data_to_analyze["is_pronoun"] == 1])}')   
print(f'pronoun without "nya": {len(data_to_analyze[data_to_analyze["is_pronoun"] == 1]) - len(list(filter(lambda x: get_markable_text(x) == "nya", data_to_analyze["id"])))}')
print(f'first person: {len(data_to_analyze[data_to_analyze["is_first_person"] == 1])}')

"nya": 23
proper name: 15
pronoun: 23
pronoun without "nya": 0
first person: 0


## Label: non-singleton, predicted: singleton

In [54]:
false_positives = wrong_predicted_markables[wrong_predicted_markables['is_singleton'].map(lambda x: x[0]) == 0]
false_positive_texts = [get_markable(idx) for idx in false_positives['id']]

In [64]:
for i, idx, false_positive_text in zip(range(1, len(false_negatives) + 1), false_positives['id'], false_positive_texts):
    print(f'{i}. {idx}. {false_positive_text}')

1. 1916. [PT ASTRA OTOPARTS TBK] menjual kepemilikan saham nya di pt exedy indonesia sebanyak <angka>
2. 1918. pt astra otoparts tbk menjual kepemilikan saham nya di [PT EXEDY INDONESIA] sebanyak <angka> saham dengan harga rp <angka> per saham atau
3. 1925. per saham atau total nya senilai rp <angka> miliar kepada [EXEDY CORPORATION] yang berkedudukan di jepang kami telah menandatangani perjanjian pengikatan untuk
4. 1927. rp <angka> miliar kepada exedy corporation yang berkedudukan di jepang [KAMI] telah menandatangani perjanjian pengikatan untuk penjualan seluruh saham milik pt
5. 1930. kami telah menandatangani perjanjian pengikatan untuk penjualan seluruh saham milik [PT ASTRA OTOPARTS] di pt exedy indonesia kepada exedy corporation kata sekretaris perusahaan
6. 1931. pengikatan untuk penjualan seluruh saham milik pt astra otoparts di [PT EXEDY INDONESIA] kepada exedy corporation kata sekretaris perusahaan astra otoparts kartina rahayu
7. 1932. saham milik pt astra otoparts di pt e

In [89]:
data_to_analyze =  false_positives

print(f'"nya": {len(list(filter(lambda x: get_markable_text(x) == "nya", data_to_analyze["id"])))}')
print(f'proper name: {len(data_to_analyze[data_to_analyze["is_proper_name"] == 1])}')
print(f'pronoun: {len(data_to_analyze[data_to_analyze["is_pronoun"] == 1])}')   
print(f'pronoun without "nya": {len(data_to_analyze[data_to_analyze["is_pronoun"] == 1]) - len(list(filter(lambda x: get_markable_text(x) == "nya", data_to_analyze["id"])))}')
print(f'first person: {len(data_to_analyze[data_to_analyze["is_first_person"] == 1])}')

"nya": 0
proper name: 23
pronoun: 2
pronoun without "nya": 2
first person: 0


# Analysis of Rightly Predicted Markables

## Non-Singletons

In [63]:
true_negatives = right_predicted_markables[right_predicted_markables['is_singleton'].map(lambda x: x[0]) == 1]
true_negative_texts = [get_markable(idx) for idx in true_negatives['id']]

In [66]:
for i, idx, true_negative_text in zip(range(1, len(true_negatives) + 1), true_negatives['id'], true_negative_texts):
    print(f'{i}. {idx}. {true_negative_text}')

1. 1917. pt astra otoparts tbk menjual kepemilikan saham [NYA] di pt exedy indonesia sebanyak <angka> saham dengan harga rp
2. 1933. astra otoparts di pt exedy indonesia kepada exedy corporation kata [SEKRETARIS PERUSAHAAN ASTRA OTOPARTS] kartina rahayu di jakarta senin menurut dia penjualan saham tersebut
3. 1934. exedy indonesia kepada exedy corporation kata sekretaris perusahaan astra otoparts [KARTINA RAHAYU] di jakarta senin menurut dia penjualan saham tersebut akan berlaku
4. 1937. sekretaris perusahaan astra otoparts kartina rahayu di jakarta senin menurut [DIA] penjualan saham tersebut akan berlaku efektif apabila syarat-syarat sebagaimana termuat
5. 1940. sebagaimana termuat dalam perjanjian pengikatan jual beli saham telah terpenuhi [DIA] mengatakan transaksi tersebut tidak mengandung unsur benturan kepentingan dilihat dari
6. 1952. <angka> persen dari pendapatan dan <angka> persen dari ekuitas tambah [NYA]
7. 1953. [MENKEU SRI MULYANI INDRAWATI] membebaskan tarif akhir bea m

In [90]:
data_to_analyze =  true_negatives

print(f'"nya": {len(list(filter(lambda x: get_markable_text(x) == "nya", data_to_analyze["id"])))}')
print(f'proper name: {len(data_to_analyze[data_to_analyze["is_proper_name"] == 1])}')
print(f'pronoun: {len(data_to_analyze[data_to_analyze["is_pronoun"] == 1])}')   
print(f'pronoun without "nya": {len(data_to_analyze[data_to_analyze["is_pronoun"] == 1]) - len(list(filter(lambda x: get_markable_text(x) == "nya", data_to_analyze["id"])))}')
print(f'first person: {len(data_to_analyze[data_to_analyze["is_first_person"] == 1])}')

"nya": 26
proper name: 40
pronoun: 37
pronoun without "nya": 11
first person: 0


## Singletons

In [67]:
true_positives = right_predicted_markables[right_predicted_markables['is_singleton'].map(lambda x: x[0]) == 0]
true_positive_texts = [get_markable(idx) for idx in true_positives['id']]

In [68]:
for i, idx, true_positive_text in zip(range(1, len(true_positives) + 1), true_positives['id'], true_positive_texts):
    print(f'{i}. {idx}. {true_positive_text}')

1. 1919. otoparts tbk menjual kepemilikan saham nya di pt exedy indonesia [SEBANYAK <ANGKA> SAHAM] dengan harga rp <angka> per saham atau total nya senilai
2. 1920. saham nya di pt exedy indonesia sebanyak <angka> saham dengan [HARGA] rp <angka> per saham atau total nya senilai rp <angka>
3. 1921. nya di pt exedy indonesia sebanyak <angka> saham dengan harga [RP <ANGKA>] per saham atau total nya senilai rp <angka> miliar kepada
4. 1922. exedy indonesia sebanyak <angka> saham dengan harga rp <angka> per [SAHAM] atau total nya senilai rp <angka> miliar kepada exedy corporation
5. 1924. saham dengan harga rp <angka> per saham atau total nya [SENILAI RP <ANGKA> MILIAR] kepada exedy corporation yang berkedudukan di jepang kami telah menandatangani
6. 1928. kepada exedy corporation yang berkedudukan di jepang kami telah menandatangani [PERJANJIAN PENGIKATAN] untuk penjualan seluruh saham milik pt astra otoparts di pt
7. 1929. berkedudukan di jepang kami telah menandatangani perjanjian pengik

In [91]:
data_to_analyze =  true_positives

print(f'"nya": {len(list(filter(lambda x: get_markable_text(x) == "nya", data_to_analyze["id"])))}')
print(f'proper name: {len(data_to_analyze[data_to_analyze["is_proper_name"] == 1])}')
print(f'pronoun: {len(data_to_analyze[data_to_analyze["is_pronoun"] == 1])}')   
print(f'pronoun without "nya": {len(data_to_analyze[data_to_analyze["is_pronoun"] == 1]) - len(list(filter(lambda x: get_markable_text(x) == "nya", data_to_analyze["id"])))}')
print(f'first person: {len(data_to_analyze[data_to_analyze["is_first_person"] == 1])}')

"nya": 0
proper name: 96
pronoun: 7
pronoun without "nya": 7
first person: 0
